In [1]:
import pandas as pd
import requests, json, time

In [2]:
# file name
filename = 'test_vcf_data.txt'

In [3]:
# check how many lines of headers are in file, where headers start with "##"

with open(filename, 'r') as file:
    
    headercount=0
    line = file.readline()
    
    while line.startswith('##'):
        headercount +=1
        line = file.readline()
        
print('Number of header lines:', headercount)

Number of header lines: 48


In [4]:
# put lines after headers into dataframe
df = pd.read_csv(filename + '.txt', sep='\t', skiprows=headercount)
df.head()

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,sample
0,1,1158631,.,A,G,2965,PASS,BRF=0.16;FR=1.0000;HP=1;HapScore=1;MGOF=3;MMLQ...,GT:GL:GOF:GQ:NR:NV,"1/1:-300.0,-43.88,0.0:3:99:160:156"
1,1,1246004,.,A,G,2965,PASS,BRF=0.09;FR=1.0000;HP=6;HapScore=1;MGOF=5;MMLQ...,GT:GL:GOF:GQ:NR:NV,"1/1:-300.0,-41.24,0.0:5:99:152:148"
2,1,1249187,.,G,A,2965,PASS,BRF=0.16;FR=1.0000;HP=3;HapScore=1;MGOF=3;MMLQ...,GT:GL:GOF:GQ:NR:NV,"1/1:-300.0,-37.63,0.0:3:99:137:135"
3,1,1261824,.,G,C,2965,PASS,BRF=0.15;FR=1.0000;HP=1;HapScore=1;MGOF=5;MMLQ...,GT:GL:GOF:GQ:NR:NV,"1/1:-300.0,-39.74,0.0:5:99:136:134"
4,1,1387667,.,C,G,2965,PASS,BRF=0.17;FR=1.0000;HP=2;HapScore=1;MGOF=3;MMLQ...,GT:GL:GOF:GQ:NR:NV,"1/1:-300.0,-39.41,0.0:3:99:137:133"


In [5]:
# separate data in FORMAT and sample columns into separate columns
# match FORMAT ID to corresponding sample value column

formatdf = pd.DataFrame(list(df.FORMAT.str.split(':')))
#print(formatdf.describe())
sampledf = pd.DataFrame(list(df['sample'].str.split(':')))
#print(sampledf.describe())

# rename columns with FORMAT ID
formatid = list(formatdf.iloc[0,:])
sampledf.columns = formatid
#sampledf.head()

In [6]:
# drop unformatted and extra columns, add in parsed column

df = df.drop(['ID','INFO','FORMAT', 'sample'], axis=1)
df = pd.concat([df, sampledf], axis=1)
#df.head()

In [7]:
# change column data types to allow later data operations

for col in ['NV', 'NR']:
    df[col] = df[col].str.replace(',','').astype(int)
for col in ['#CHROM','REF', 'ALT']:
    df[col] = df[col].astype(str)

In [8]:
# generate column for percentage reads supporting variant

df['NV/NR'] = df['NV']/df['NR']

In [9]:
# function to determine type of variant
# inputs: reference base(s) and alternate base(s) from VCF
# outputs: type of variant
# currently, only identifies some variant types

def get_variant_type(REF, ALT):
    if (len(REF) == 1) and (len(ALT) == 1):
        vartype = 'substitution'
        
    elif (len(REF) == 1) and (len(ALT) > 1):
        vartype = 'insertion'
            
    elif len(REF) > len(ALT):
        vartype = 'deletion'
        
    else:
        # unknown variant type
        vartype = 'unknown'
        
    return vartype

In [10]:
# function to put variant into hgvs notation
# inputs:  chromosome, variant position, reference base(s), and alternate base(s) from VCF, 
# and variant type (from function get_variant_type)
# outputs: hgvs notation
# currently, only some variant types covered

def get_hgvs_notation(chromosome, position, REF, ALT, varianttype):
    # base
    hgvs_part1 = chromosome + ':' + 'g.' + str(position)
    
    # variant specific
    if varianttype == 'substitution':
        hgvs_part2 = REF + '>' + ALT
        
    elif varianttype == 'insertion':
        hgvs_part2 = '_' + str(position + 1) + 'ins' + ALT[1:]
        
    elif varianttype == 'deletion':
        hgvs_part2 = '_' + str(position + len(REF) - 1) + 'del'
        
    else:
        # unknown variant type
        hgvs_part2 = ''
        
    if hgvs_part2 == '':
        # put placeholder, ideally, all variant types put into proper hgvs notation
        hgvs_notation = '1:g.25362501C>A'
    else:
        hgvs_notation = hgvs_part1 + hgvs_part2
    
    return hgvs_notation

In [11]:
# function to process Ensembl VEP hgvs API response from POST request
# input: response from API as dict, processed by json.loads()
# outputs: lists of gene symbol, most severe consequence, and minor allele frequency for each variant in input data

def get_info(data):
    gene = []
    effect = []
    minoralfreq = []

    for item in range(len(data)):
        # get gene, if available
        if 'transcript_consequences' in data[item] \
        and 'gene_symbol' in data[item]['transcript_consequences'][0]:
            gene.append(data[item]['transcript_consequences'][0]['gene_symbol'])
        else:
            gene.append('unknown')

        # get effect, if available
        if 'most_severe_consequence' in data[item]:
            effect.append(data[item]['most_severe_consequence'])
        else:
            effect.append('unknown')

        # get minor allele frequency, if available
        if 'colocated_variants' in data[item] \
        and ('minor_allele_freq' in data[item]['colocated_variants'][0]):
            minoralfreq.append(data[item]['colocated_variants'][0]['minor_allele_freq'])

        elif 'colocated_variants' in data[item] \
        and (len(data[item]['colocated_variants']) > 1) \
        and ('minor_allele_freq' in data[item]['colocated_variants'][1]):
            minoralfreq.append(data[item]['colocated_variants'][1]['minor_allele_freq'])
        else:
            minoralfreq.append('unknown')
            
    return gene, effect, minoralfreq

In [12]:
# query api and get data

count = 0
typelist = []
varlist = [] 
genelist = []
effectlist = []
minoralfreqlist = []

for variant in range(len(df)):
    
    REF = df.loc[variant, 'REF'] # reference base(s)
    ALT = df.loc[variant, 'ALT'] # alternate base(s)
    POS = df.loc[variant, 'POS'] # variant position
    CHROM = df.loc[variant, '#CHROM'] # chromosome
    
    # get type of variant
    vartype = get_variant_type(REF, ALT)
    typelist.append(vartype)
    
    # put variant into hgvs notation, make hgvs list for api query
    varlist.append(get_hgvs_notation(CHROM, POS, REF, ALT, vartype))
    
    if len(varlist) >= 300 or variant == (len(df)-1):
        # just to see progress or lack of
        count += 1
        print('batch: ', count)
            
        # query api
        # since reference sequence is based off GRCh37, connect to grch37 site
        url = 'https://grch37.rest.ensembl.org'
        endpoint = '/vep/human/hgvs'
        headers = {'Content-Type':'application/json', 'Accept':'application/json'}
        hgvs_list = {'hgvs_notations': varlist}


        # make request
        for retry in range(5):
            try:
                print('make request...')
                r = requests.post(url + endpoint, headers=headers, data=json.dumps(hgvs_list))
                break

            except requests.ConnectionError:
                time.sleep(1)
                continue

        # get data if request successful
        if r.status_code == 200:
            print('load response...')
            data = json.loads(r.text)
            
            # get info from response
            gene, effect, minoralfreq = get_info(data)
            
            # join lists gathered from each response
            genelist = genelist + gene
            effectlist = effectlist + effect
            minoralfreqlist = minoralfreqlist + minoralfreq
            
            # reset to empty for next batch    
            varlist = []
        else:
            print(r.status_code)
            print(r.headers)
            
        
    else:
        continue
        
#print(typelist)
#print(genelist)
#print(effectlist)
#print(minoralfreqlist)

batch:  1
make request...
load response...
batch:  2
make request...
load response...
batch:  3
make request...
load response...
batch:  4
make request...
load response...
batch:  5
make request...
load response...
batch:  6
make request...
load response...
batch:  7
make request...
load response...
batch:  8
make request...
load response...
batch:  9
make request...
load response...
batch:  10
make request...
load response...
batch:  11
make request...
load response...
batch:  12
make request...
load response...
batch:  13
make request...
load response...
batch:  14
make request...
load response...
batch:  15
make request...
load response...
batch:  16
make request...
load response...
batch:  17
make request...
load response...
batch:  18
make request...
load response...
batch:  19
make request...
load response...
batch:  20
make request...
load response...
batch:  21
make request...
load response...
batch:  22
make request...
load response...
batch:  23
make request...
load response.

In [13]:
# fill in dataframe
df['gene'] = genelist
df['type'] = typelist
df['effect'] = effectlist
df['minor_allele_freq'] = minoralfreqlist


In [14]:
# clean up dataframe a bit
df = df.drop(['GT', 'GL','GOF', 'GQ'], axis=1)
df['effect'] = df['effect'].str.replace('_',' ')
df = df.rename(columns={'#CHROM': 'CHROM'})

In [15]:
# save to csv
df.to_csv(filename.split('.')[0] + '_annotated.csv')